In [47]:
import os
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import lxml
import numbers
from bs4 import BeautifulSoup
import requests
import json

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [48]:
page_url = 'https://fortune.com/wp-json/irving/v1/data/franchise-search-results?list_id=2013055&token=Zm9ydHVuZTpCcHNyZmtNZCN5SndjWkkhNHFqMndEOTM='

In [49]:
# request the URL and parse the JSON
response = requests.get(page_url)
response.raise_for_status() # raise exception if invalid response
comps = response.json()
print(len(comps))

2


In [50]:
comp = comps[1]
print(len(comp))

2


In [51]:
# prettify json
format_json = json.dumps(comp, sort_keys=True, indent=5)
print(format_json[:1000])

{
     "items": [
          {
               "fields": [
                    {
                         "key": "rank",
                         "value": "351"
                    },
                    {
                         "key": "revenues",
                         "value": "7710"
                    },
                    {
                         "key": "revchange",
                         "value": "-5.1"
                    },
                    {
                         "key": "profits",
                         "value": "119.9"
                    },
                    {
                         "key": "prftchange",
                         "value": ""
                    },
                    {
                         "key": "assets",
                         "value": "2606"
                    },
                    {
                         "key": "mktval",
                         "value": "1786"
                    },
                    {
                     

In [52]:
# select the third company
n_comp = comp['items'][2]
print(n_comp)

{'fields': [{'key': 'rank', 'value': '353'}, {'key': 'revenues', 'value': '7669'}, {'key': 'revchange', 'value': '28.2'}, {'key': 'profits', 'value': '602.7'}, {'key': 'prftchange', 'value': '22'}, {'key': 'assets', 'value': '10178'}, {'key': 'mktval', 'value': '7466'}, {'key': 'employees', 'value': '4623'}, {'key': 'rankchange1000', 'value': '80'}, {'key': 'rankchange', 'value': '80'}, {'fieldMeta': {'title': 'Company', 'type': 'Text', 'importField': 'title', 'order': 'desc'}, 'key': 'title', 'value': 'PulteGroup'}], 'permalink': 'https://fortune.com/fortune500/2017/pultegroup/'}


In [53]:
n_comp.keys()

dict_keys(['fields', 'permalink'])

In [54]:
new_comp = n_comp['permalink']
print(new_comp)

https://fortune.com/fortune500/2017/pultegroup/


In [55]:
my_comp = n_comp['fields']
print(my_comp)

[{'key': 'rank', 'value': '353'}, {'key': 'revenues', 'value': '7669'}, {'key': 'revchange', 'value': '28.2'}, {'key': 'profits', 'value': '602.7'}, {'key': 'prftchange', 'value': '22'}, {'key': 'assets', 'value': '10178'}, {'key': 'mktval', 'value': '7466'}, {'key': 'employees', 'value': '4623'}, {'key': 'rankchange1000', 'value': '80'}, {'key': 'rankchange', 'value': '80'}, {'fieldMeta': {'title': 'Company', 'type': 'Text', 'importField': 'title', 'order': 'desc'}, 'key': 'title', 'value': 'PulteGroup'}]


# JSON Data

In [56]:
fields = []

for j in comp['items']:

    #for key, value in comp:
    field = j['fields']
    fields.append(field)

print(len(fields))

1000


In [57]:
df = pd.DataFrame()

In [58]:
def gather_data(name, col):

    lst = []
    val = ""
    for i in fields:
        for j in i:
            if j['key'] == name:
                val = j['value']
            else:
                pass

        lst.append(val)

    df[col] = lst

    return df

In [59]:
keys = ['name', 'rank', 'rankchange', 'f500_revenues', 'f500_profits',
        'f500_ employees', 'sector', 'hqcity', 'hqstate', 'newcomer',
        'ceofounder', 'ceowoman', 'profitable']

cols = ['company', 'rank', 'rank_change', 'revenue', 'profit',
        'num. of employees', 'sector', 'city', 'state', 'newcomer',
        'ceo_founder', 'ceo_woman', 'profitable']


for x,y in zip(keys, cols):
    f500_df = gather_data(x, y)

In [60]:
print(f500_df.shape)
f500_df.head()

(1000, 13)


,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable
0,,351,-15,,,,,,,,,,
1,,352,79,,,,,,,,,,
2,,353,80,,,,,,,,,,
3,,354,14,,,,,,,,,,
4,,355,-3,,,,,,,,,,


In [61]:
f500_df[['rank', 'rank_change', 'revenue', 'profit','num. of employees']] = f500_df[['rank', 'rank_change', 'revenue','profit','num. of employees']].apply(pd.to_numeric)

In [62]:
# fill missing values with zero
f500_df['rank_change'] = f500_df['rank_change'].fillna(0)

In [63]:
print(f500_df.dtypes)

company               object
rank                   int64
rank_change          float64
revenue              float64
profit               float64
num. of employees    float64
sector                object
city                  object
state                 object
newcomer              object
ceo_founder           object
ceo_woman             object
profitable            object
dtype: object


In [64]:
# Create new features
f500_df['prev_rank'] = (f500_df['rank'] + f500_df['rank_change'])
f500_df.head()

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank
0,,351,-15.0,NaN,NaN,NaN,,,,,,,,336.0
1,,352,79.0,NaN,NaN,NaN,,,,,,,,431.0
2,,353,80.0,NaN,NaN,NaN,,,,,,,,433.0
3,,354,14.0,NaN,NaN,NaN,,,,,,,,368.0
4,,355,-3.0,NaN,NaN,NaN,,,,,,,,352.0


In [65]:
# newcommer prev_rank
f500_df.loc[(f500_df['newcomer'] == 'yes')| (f500_df['rank']>499), 'prev_rank'] = " "

In [68]:
f500_df

,company,rank,rank_change,revenue,profit,num. of employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank
0,,351,-15.0,NaN,NaN,NaN,,,,,,,,336.0
1,,352,79.0,NaN,NaN,NaN,,,,,,,,431.0
2,,353,80.0,NaN,NaN,NaN,,,,,,,,433.0
3,,354,14.0,NaN,NaN,NaN,,,,,,,,368.0
4,,355,-3.0,NaN,NaN,NaN,,,,,,,,352.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,,996,0.0,NaN,NaN,NaN,,,,,,,,
996,,997,0.0,NaN,NaN,NaN,,,,,,,,
997,,998,0.0,NaN,NaN,NaN,,,,,,,,
998,,999,0.0,NaN,NaN,NaN,,,,,,,,


In [67]:
f500_df.to_csv("2017(2).csv", index=False)

# Crawl URL

# Join and sort